<a href="https://colab.research.google.com/github/nh0127/SEF_DB/blob/main/SEF_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

식품의약품안전처_의약품 제품 허가정보_의약품 제품 허가 상세 정보

In [ ]:
# (상단 코드 및 CipherSuiteAdapter 클래스는 이전과 동일)
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_all_drug_data_in_one_go(service_key, save_filename="drug_permission_data_final.json"):
    """
    일일 트래픽(10,000회)이 충분하므로, 하루 만에 모든 데이터를 수집하는 최종 함수.
    """
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtPrmsnDtlInq05"
    full_api_url = api_url_base + api_path
    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # 1. 1페이지를 요청하여 전체 데이터 수를 정확히 파악
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False
        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False

    # 2. 1페이지부터 마지막 페이지까지 순차적으로 수집
    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            if data['header']['resultCode'] == '00' and 'items' in data['body'] and data['body']['items']:
                items_data = data['body']['items']
                if not isinstance(items_data, list): items_data = [items_data]
                all_items.extend(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', '데이터 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이
            time.sleep(0.2)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    # 3. 최종 저장
    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)
        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

# --- 스크립트 실행 ---
if __name__ == "__main__":
    MY_SERVICE_KEY = "k5pE3X4v37ySjqiAzhxMCS8PbYK6S7G7J8iWiiJXUs2LpCcENDTIHsQ+t5quezvkY3dgGn8OOteFJOWZkxw3Ow=="
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

데이터 수집을 시작합니다...
총 데이터: 44248개, 전체 페이지: 443개를 수집합니다.
--------------------------------------------------
페이지 1/443 수집 중...
페이지 2/443 수집 중...
페이지 3/443 수집 중...
페이지 4/443 수집 중...
페이지 5/443 수집 중...
페이지 6/443 수집 중...
페이지 7/443 수집 중...
페이지 8/443 수집 중...
페이지 9/443 수집 중...
페이지 10/443 수집 중...
페이지 11/443 수집 중...
페이지 12/443 수집 중...
페이지 13/443 수집 중...
페이지 14/443 수집 중...
페이지 15/443 수집 중...
페이지 16/443 수집 중...
페이지 17/443 수집 중...
페이지 18/443 수집 중...
페이지 19/443 수집 중...
페이지 20/443 수집 중...
페이지 21/443 수집 중...
페이지 22/443 수집 중...
페이지 23/443 수집 중...
페이지 24/443 수집 중...
페이지 25/443 수집 중...
페이지 26/443 수집 중...
페이지 27/443 수집 중...
페이지 28/443 수집 중...
페이지 29/443 수집 중...
페이지 30/443 수집 중...
페이지 31/443 수집 중...
페이지 32/443 수집 중...
페이지 33/443 수집 중...
페이지 34/443 수집 중...
페이지 35/443 수집 중...
페이지 36/443 수집 중...
페이지 37/443 수집 중...
페이지 38/443 수집 중...
페이지 39/443 수집 중...
페이지 40/443 수집 중...
페이지 41/443 수집 중...
페이지 42/443 수집 중...
페이지 43/443 수집 중...
페이지 44/443 수집 중...
페이지 45/443 수집 중...
페이지 46/443 수집 중...
페이지 47/443 수집 중...
페이지 48/443 수

페이지 107/443 수집 중...
페이지 108/443 수집 중...
페이지 109/443 수집 중...
페이지 110/443 수집 중...
페이지 111/443 수집 중...
페이지 112/443 수집 중...
페이지 113/443 수집 중...
페이지 114/443 수집 중...
페이지 115/443 수집 중...
페이지 116/443 수집 중...
페이지 117/443 수집 중...
페이지 118/443 수집 중...
페이지 119/443 수집 중...
페이지 120/443 수집 중...
페이지 121/443 수집 중...
페이지 122/443 수집 중...
페이지 123/443 수집 중...
페이지 124/443 수집 중...
페이지 125/443 수집 중...
페이지 126/443 수집 중...
페이지 127/443 수집 중...
페이지 128/443 수집 중...
페이지 129/443 수집 중...
페이지 130/443 수집 중...
페이지 131/443 수집 중...
페이지 132/443 수집 중...
페이지 133/443 수집 중...
페이지 134/443 수집 중...
페이지 135/443 수집 중...
페이지 136/443 수집 중...
페이지 137/443 수집 중...
페이지 138/443 수집 중...
페이지 139/443 수집 중...
페이지 140/443 수집 중...
페이지 141/443 수집 중...
페이지 142/443 수집 중...
페이지 143/443 수집 중...
페이지 144/443 수집 중...
페이지 145/443 수집 중...
페이지 146/443 수집 중...
페이지 147/443 수집 중...
페이지 148/443 수집 중...
페이지 149/443 수집 중...
페이지 150/443 수집 중...
페이지 151/443 수집 중...
페이지 152/443 수집 중...
페이지 153/443 수집 중...
페이지 154/443 수집 중...
페이지 155/443 수집 중...
페이지 156/443 수집 중...


페이지 182/443 수집 중...
페이지 183/443 수집 중...
페이지 184/443 수집 중...
페이지 185/443 수집 중...
페이지 186/443 수집 중...
페이지 187/443 수집 중...
페이지 188/443 수집 중...
페이지 189/443 수집 중...
페이지 190/443 수집 중...
페이지 191/443 수집 중...
페이지 192/443 수집 중...
페이지 193/443 수집 중...
페이지 194/443 수집 중...
페이지 195/443 수집 중...
페이지 196/443 수집 중...
페이지 197/443 수집 중...
페이지 198/443 수집 중...
페이지 199/443 수집 중...
페이지 200/443 수집 중...
  -> 200 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 201/443 수집 중...
페이지 202/443 수집 중...
페이지 203/443 수집 중...
페이지 204/443 수집 중...
페이지 205/443 수집 중...
페이지 206/443 수집 중...
페이지 207/443 수집 중...
페이지 208/443 수집 중...
페이지 209/443 수집 중...
페이지 210/443 수집 중...
페이지 211/443 수집 중...
페이지 212/443 수집 중...
페이지 213/443 수집 중...
페이지 214/443 수집 중...
페이지 215/443 수집 중...
페이지 216/443 수집 중...
페이지 217/443 수집 중...
페이지 218/443 수집 중...
페이지 219/443 수집 중...
페이지 220/443 수집 중...
페이지 221/443 수집 중...
페이지 222/443 수집 중...
페이지 223/443 수집 중...
페이지 224/443 수집 중...
페이지 225/443 수집 중...
페이지 226/443 수집 중...
페이지 227/443 수집 중...
페이지 228/443 수집 중...
페이지 229/443 수집 중

페이지 262/443 수집 중...
페이지 263/443 수집 중...
페이지 264/443 수집 중...
페이지 265/443 수집 중...
페이지 266/443 수집 중...
페이지 267/443 수집 중...
페이지 268/443 수집 중...
페이지 269/443 수집 중...
페이지 270/443 수집 중...
페이지 271/443 수집 중...
페이지 272/443 수집 중...
페이지 273/443 수집 중...
페이지 274/443 수집 중...
페이지 275/443 수집 중...
페이지 276/443 수집 중...
페이지 277/443 수집 중...
페이지 278/443 수집 중...
페이지 279/443 수집 중...
페이지 280/443 수집 중...
페이지 281/443 수집 중...
페이지 282/443 수집 중...
페이지 283/443 수집 중...
페이지 284/443 수집 중...
페이지 285/443 수집 중...
페이지 286/443 수집 중...
페이지 287/443 수집 중...
페이지 288/443 수집 중...
페이지 289/443 수집 중...
페이지 290/443 수집 중...
페이지 291/443 수집 중...
페이지 292/443 수집 중...
페이지 293/443 수집 중...
페이지 294/443 수집 중...
페이지 295/443 수집 중...
페이지 296/443 수집 중...
페이지 297/443 수집 중...
페이지 298/443 수집 중...
페이지 299/443 수집 중...
페이지 300/443 수집 중...
  -> 300 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 301/443 수집 중...
페이지 302/443 수집 중...
페이지 303/443 수집 중...
페이지 304/443 수집 중...
페이지 305/443 수집 중...
페이지 306/443 수집 중...
페이지 307/443 수집 중...
페이지 308/443 수집 중...
페이지 309/443 수집 중

페이지 349/443 수집 중...
페이지 350/443 수집 중...
페이지 351/443 수집 중...
페이지 352/443 수집 중...
페이지 353/443 수집 중...
페이지 354/443 수집 중...
페이지 355/443 수집 중...
페이지 356/443 수집 중...


페이지 357/443 수집 중...
페이지 358/443 수집 중...
페이지 359/443 수집 중...
페이지 360/443 수집 중...
페이지 361/443 수집 중...
페이지 362/443 수집 중...
페이지 363/443 수집 중...
  -> !!! 페이지 363 수집 실패: ('Connection broken: IncompleteRead(961 bytes read, 9279 more expected)', IncompleteRead(961 bytes read, 9279 more expected)). 이 페이지를 건너뜁니다.
페이지 364/443 수집 중...
페이지 365/443 수집 중...
페이지 366/443 수집 중...
페이지 367/443 수집 중...
페이지 368/443 수집 중...
페이지 369/443 수집 중...
페이지 370/443 수집 중...
페이지 371/443 수집 중...
페이지 372/443 수집 중...
페이지 373/443 수집 중...
페이지 374/443 수집 중...
페이지 375/443 수집 중...
페이지 376/443 수집 중...
페이지 377/443 수집 중...
페이지 378/443 수집 중...
페이지 379/443 수집 중...
페이지 380/443 수집 중...
페이지 381/443 수집 중...


페이지 382/443 수집 중...
페이지 383/443 수집 중...
페이지 384/443 수집 중...
페이지 385/443 수집 중...
페이지 386/443 수집 중...
페이지 387/443 수집 중...
페이지 388/443 수집 중...
페이지 389/443 수집 중...
페이지 390/443 수집 중...
페이지 391/443 수집 중...


페이지 392/443 수집 중...
페이지 393/443 수집 중...
페이지 394/443 수집 중...
페이지 395/443 수집 중...
페이지 396/443 수집 중...
페이지 397/443 수집 중...
페이지 398/443 수집 중...
페이지 399/443 수집 중...
페이지 400/443 수집 중...
  -> 400 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...
페이지 401/443 수집 중...
페이지 402/443 수집 중...
페이지 403/443 수집 중...
페이지 404/443 수집 중...
페이지 405/443 수집 중...
페이지 406/443 수집 중...
페이지 407/443 수집 중...
페이지 408/443 수집 중...
페이지 409/443 수집 중...
페이지 410/443 수집 중...
페이지 411/443 수집 중...
페이지 412/443 수집 중...
페이지 413/443 수집 중...
페이지 414/443 수집 중...


페이지 415/443 수집 중...
페이지 416/443 수집 중...
페이지 417/443 수집 중...
페이지 418/443 수집 중...
페이지 419/443 수집 중...
페이지 420/443 수집 중...
페이지 421/443 수집 중...
페이지 422/443 수집 중...
페이지 423/443 수집 중...
페이지 424/443 수집 중...
페이지 425/443 수집 중...
페이지 426/443 수집 중...
페이지 427/443 수집 중...
페이지 428/443 수집 중...
페이지 429/443 수집 중...
페이지 430/443 수집 중...
페이지 431/443 수집 중...
페이지 432/443 수집 중...
페이지 433/443 수집 중...
페이지 434/443 수집 중...
페이지 435/443 수집 중...
페이지 436/443 수집 중...
페이지 437/443 수집 중...
페이지 438/443 수집 중...
페이지 439/443 수집 중...
페이지 440/443 수집 중...
페이지 441/443 수집 중...
페이지 442/443 수집 중...
페이지 443/443 수집 중...
--------------------------------------------------
총 44148개의 데이터를 성공적으로 수집했습니다.
'drug_permission_data_final.json' 파일로 최종 저장합니다...
저장 완료! 모든 데이터 수집이 완료되었습니다!


In [ ]:
import pandas as pd
import json
import re

print("새로운 데이터 전처리를 시작합니다...")

# 1단계: 파일 로드
try:
    df = pd.read_json('drug_permission_data_final.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_permission_data_final.json 파일을 찾을 수 없습니다. 파일이 현재 폴더에 있는지 확인하세요.")
    exit() # 파일이 없으면 종료

# 2단계: 기본 정제
# ------------------------------------------------------------------------------
# 2.1. 컬럼 이름을 소문자로 통일 (e.g., ITEM_NAME -> item_name)
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

# 2.2. 고유 ID(item_seq) 기준으로 중복 데이터 제거
before_dedup = len(df)
df.drop_duplicates(subset=['item_seq'], keep='first', inplace=True)
after_dedup = len(df)
print(f"중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

# 2.3. 주요 텍스트 컬럼의 결측값(NaN)을 빈 문자열('')로 처리
text_cols = ['ee_doc_data', 'ud_doc_data', 'nb_doc_data', 'ingr_name']
df[text_cols] = df[text_cols].fillna('')
print("결측값 처리를 완료했습니다.")

# 3단계: 정보 구조화 및 표준화 (Feature Engineering)
# ------------------------------------------------------------------------------

# 3.1. 제품명(item_name)에서 괄호와 성분명 제거
def remove_brackets(name):
    """제품명에서 괄호와 그 안의 내용만 제거하는 함수"""
    cleaned_name = name
    # 중첩 괄호를 처리하기 위해 5번 반복 (대부분의 경우 충분)
    for _ in range(5):
        if '(' not in cleaned_name: break
        cleaned_name = re.sub(r'\s*\([^)]*\)', '', cleaned_name)
    return cleaned_name.strip()

df['item_name_clean'] = df['item_name'].apply(remove_brackets)
print("\n제품명(item_name) 정제를 완료했습니다.")

# 3.2. 성분명(ingr_name)을 파싱하여 리스트로 만들기
def parse_ingredients(text):
    """쉼표(,)나 세미콜론(;)으로 구분된 성분명을 리스트로 변환하는 함수"""
    if not text:
        return []
    # 구분자를 기준으로 분리하고 각 성분의 앞뒤 공백 제거
    ingredients = [ing.strip() for ing in re.split(r'[,;]', text)]
    # 빈 문자열은 리스트에서 제외
    return [ing for ing in ingredients if ing]

df['ingredients_list'] = df['ingr_name'].apply(parse_ingredients)
print("성분명(ingr_name) 파싱을 완료했습니다.")

# 3.3. 주의사항(nb_doc_data)에서 상호작용 약물 목록 추출
def parse_interactions_from_notes(text):
    """주의사항 텍스트에서 상호작용 약물 목록을 추출하는 함수"""
    if '상호작용' not in text or '병용' not in text:
        return []

    # 다양한 목록 형태(1., 1), •, -)를 처리하는 정규표현식
    pattern = r'(?:[①-⑨\d][\.\)]|•|-)\s*([가-힣a-zA-Z0-9\s/]+?)(?=[,;\n①-⑨\d•-]|$)'
    drugs = re.findall(pattern, text)

    cleaned_drugs = []
    for drug in drugs:
        clean_drug = drug.strip().split(',')[0].strip()
        clean_drug = re.sub(r'\s+등$', '', clean_drug)
        if clean_drug:
            cleaned_drugs.append(clean_drug)

    return cleaned_drugs

df['interactions_list'] = df['nb_doc_data'].apply(parse_interactions_from_notes)
print("주의사항(nb_doc_data)에서 상호작용 정보 추출을 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 챗봇에 필요한 최종 컬럼들 선택
final_v2_df = df[[
    'item_seq',
    'item_name',
    'item_name_clean', # 정제된 제품명
    'entp_name',
    'etc_otc_code', # 전문/일반의약품 코드
    'ingr_name', # 원본 성분명 텍스트
    'ingredients_list', # 파싱된 성분명 리스트
    'ee_doc_data', # 효능효과
    'ud_doc_data', # 용법용량
    'nb_doc_data', # 주의사항 원본 텍스트
    'interactions_list' # 추출된 상호작용 리스트
]]

# 효율적인 Parquet 포맷으로 저장
try:
    save_filename = 'drug_data_processed_v2.parquet'
    final_v2_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    # pyarrow 라이브러리가 없는 경우 CSV로 저장
    save_filename = 'drug_data_processed_v2.csv'
    final_v2_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print("\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 (상호작용 정보가 있는 데이터 위주) ---")
result_check = final_v2_df[final_v2_df['interactions_list'].apply(len) > 0]
if not result_check.empty:
    print(result_check[['item_name_clean', 'ingredients_list', 'interactions_list']].head())
else:
    print("상호작용 정보를 포함한 데이터를 찾지 못했습니다. 원본 데이터를 확인해보세요.")

새로운 데이터 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (44148, 42)

컬럼 이름을 소문자로 변경했습니다.
중복 제거: 44148개 -> 44144개 (4개 제거)
결측값 처리를 완료했습니다.

제품명(item_name) 정제를 완료했습니다.
성분명(ingr_name) 파싱을 완료했습니다.
주의사항(nb_doc_data)에서 상호작용 정보 추출을 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v2.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 (상호작용 정보가 있는 데이터 위주) ---
          item_name_clean                 ingredients_list  \
9          제일에페드린염산염주사액4%                  [[M040534]주사용수]   
11              안나카주사액20%                  [[M040534]주사용수]   
12              안나카주사액10%                  [[M040534]주사용수]   
14  대원염산에페드린주사액[수출명:에린주사]                  [[M040534]주사용수]   
15             대원아미노필린주사액  [[M040534]주사용수|[M223366]에틸렌디아민]   

                                    interactions_list  
9   [MAO억제제를 투여중이거나 투여를 중지한 지, 갑상샘기능항진증, 고량투여 시 불안...  
11                                 [MAO억제제와 병용투여시 빈맥]  
12                                 [MAO억제제와 병용투여시 빈맥]  
14  [MAO억제제를 투여중이거나 투여를 중지한 지, 갑상샘기능항진증, 고량투여 시 불안...  
15  [앰플주사제는 용기절단시 유리파편이 혼입

식품의약품안전처_의약품 제품 허가정보_의약품 제품 허가 목록

In [ ]:
# 이전에 드렸던 최종 완성 코드와 동일합니다.
# 이 코드를 다시 실행해주세요.
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_drug_permit_list_data(service_key, save_filename="drug_permit_list_data.json"):
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtPrmsnInq06"
    full_api_url = api_url_base + api_path

    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    print("새로운 데이터('의약품 제품 허가 목록') 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False

        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False

    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            if (data['header']['resultCode'] == '00' and 'items' in data['body']
                and data['body']['items']): # 'item' 키가 없을 수도 있으므로 items 까지만 확인

                items_data = data['body']['items'] # items 에 바로 리스트가 올 수도 있음
                if 'item' in items_data: # item 키가 있으면 한 단계 더 들어감
                    items_data = items_data['item']

                if isinstance(items_data, list):
                    all_items.extend(items_data)
                elif isinstance(items_data, dict):
                    all_items.append(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', 'items 키 없음')}")

            time.sleep(0.2)

            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")

        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)

        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

if __name__ == "__main__":
    MY_SERVICE_KEY = "k5pE3X4v37ySjqiAzhxMCS8PbYK6S7G7J8iWiiJXUs2LpCcENDTIHsQ+t5quezvkY3dgGn8OOteFJOWZkxw3Ow=="
    download_drug_permit_list_data(service_key=MY_SERVICE_KEY)

새로운 데이터('의약품 제품 허가 목록') 수집을 시작합니다...
총 데이터: 44248개, 전체 페이지: 443개를 수집합니다.
--------------------------------------------------
페이지 1/443 수집 중...
페이지 2/443 수집 중...
페이지 3/443 수집 중...
페이지 4/443 수집 중...
페이지 5/443 수집 중...
페이지 6/443 수집 중...
페이지 7/443 수집 중...
페이지 8/443 수집 중...
페이지 9/443 수집 중...
페이지 10/443 수집 중...
페이지 11/443 수집 중...
페이지 12/443 수집 중...
페이지 13/443 수집 중...
페이지 14/443 수집 중...
페이지 15/443 수집 중...
페이지 16/443 수집 중...
페이지 17/443 수집 중...
페이지 18/443 수집 중...
페이지 19/443 수집 중...
페이지 20/443 수집 중...
페이지 21/443 수집 중...
페이지 22/443 수집 중...
페이지 23/443 수집 중...
페이지 24/443 수집 중...
페이지 25/443 수집 중...
페이지 26/443 수집 중...
페이지 27/443 수집 중...
페이지 28/443 수집 중...
페이지 29/443 수집 중...
페이지 30/443 수집 중...
페이지 31/443 수집 중...
페이지 32/443 수집 중...
페이지 33/443 수집 중...
페이지 34/443 수집 중...
페이지 35/443 수집 중...
페이지 36/443 수집 중...
페이지 37/443 수집 중...
페이지 38/443 수집 중...
페이지 39/443 수집 중...
페이지 40/443 수집 중...
페이지 41/443 수집 중...
페이지 42/443 수집 중...
페이지 43/443 수집 중...
페이지 44/443 수집 중...
페이지 45/443 수집 중...
페이지 46/443 수집 중...
페이지 47/443 

In [ ]:
import pandas as pd
import json
import re

print("새로운 데이터('의약품 제품 허가 목록') 전처리를 시작합니다...")

# 1단계: 파일 로드
try:
    df = pd.read_json('drug_permit_list_data.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_permit_list_data.json 파일을 찾을 수 없습니다. 파일이 현재 폴더에 있는지 확인하세요.")
    exit()

# 2단계: 기본 정제
# ------------------------------------------------------------------------------
# 2.1. 컬럼 이름을 스네이크 케이스로 통일 (일관성 유지)
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

# 2.2. 고유 ID(item_seq) 기준으로 중복 데이터 제거
before_dedup = len(df)
df.drop_duplicates(subset=['item_seq'], keep='first', inplace=True)
after_dedup = len(df)
print(f"중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

# 2.3. 주요 텍스트 컬럼의 결측값(NaN)을 빈 문자열('')로 처리
text_cols = ['item_eng_name', 'entp_eng_name', 'item_ingr_name', 'cancel_name']
df[text_cols] = df[text_cols].fillna('')
print("결측값 처리를 완료했습니다.")

# 3단계: 정보 구조화 및 표준화 (Feature Engineering)
# ------------------------------------------------------------------------------

# 3.1. 제품명(item_name)에서 괄호와 성분명 제거
def remove_brackets(name):
    """제품명에서 괄호와 그 안의 내용만 제거하는 함수"""
    if not isinstance(name, str): return "" # 문자열이 아닌 경우 대비
    cleaned_name = name
    for _ in range(5):
        if '(' not in cleaned_name: break
        cleaned_name = re.sub(r'\s*\([^)]*\)', '', cleaned_name)
    return cleaned_name.strip()

df['item_name_clean'] = df['item_name'].apply(remove_brackets)
print("\n제품명(item_name) 정제를 완료했습니다.")

# 3.2. 주성분명(item_ingr_name)을 파싱하여 리스트로 만들기
#    이 데이터의 주성분명은 '성분1,성분2, ...' 형태로 되어 있을 가능성이 높습니다.
def parse_main_ingredients(text):
    """쉼표(,)로 구분된 주성분명을 리스트로 변환하는 함수"""
    if not text or not isinstance(text, str):
        return []
    # 구분자를 기준으로 분리하고 각 성분의 앞뒤 공백 제거
    ingredients = [ing.strip() for ing in text.split(',')]
    # 빈 문자열은 리스트에서 제외
    return [ing for ing in ingredients if ing]

df['main_ingredients_list'] = df['item_ingr_name'].apply(parse_main_ingredients)
print("주성분명(item_ingr_name) 파싱을 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 이 데이터에서 유용하게 사용할 컬럼들 선택
final_df = df[[
    'item_seq',
    'item_name',
    'item_name_clean', # 정제된 제품명
    'entp_name',
    'spclty_pblc', # 전문/일반의약품 구분
    'prduct_type', # 제품 타입 (예: 완제의약품)
    'item_ingr_name', # 원본 주성분 텍스트
    'main_ingredients_list', # 파싱된 주성분 리스트
    'big_prdt_img_url', # 제품 이미지 URL
    'edi_code' # EDI 코드 (보험코드)
]]

# 효율적인 Parquet 포맷으로 저장
try:
    save_filename = 'drug_data_processed_v3.parquet'
    final_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v3.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 ---")
print(final_df[['item_name_clean', 'spclty_pblc', 'main_ingredients_list']].head())

새로운 데이터('의약품 제품 허가 목록') 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (44248, 21)

컬럼 이름을 소문자로 변경했습니다.
중복 제거: 44248개 -> 44244개 (4개 제거)
결측값 처리를 완료했습니다.

제품명(item_name) 정제를 완료했습니다.
주성분명(item_ingr_name) 파싱을 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v3.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 ---
  item_name_clean spclty_pblc      main_ingredients_list
0    중외5%포도당생리식염액       전문의약품  [Glucose/Sodium Chloride]
1      중외5%포도당주사액       전문의약품                  [Glucose]
2     중외20%포도당주사액       전문의약품                  [Glucose]
3     중외50%포도당주사액       전문의약품                  [Glucose]
4        대한포도당주사액       전문의약품                  [Glucose]


식품의약품안전처_의약품 제품 허가정보_의약품 제품 주성분 상세정보

In [ ]:
import requests
import json
import math
import time
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# SSL/TLS 암호화 스위트 관련 문제를 해결하기 위한 어댑터 클래스
# data.go.kr 서버와 통신 시 간혹 발생하는 SSL 오류를 우회합니다.
class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        # 특정 암호화 스위트 목록을 지정합니다.
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:ECDH+HIGH:'
                   'DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False, **pool_kwargs):
        # 커스텀 SSL 컨텍스트를 PoolManager에 적용합니다.
        pool_kwargs['ssl_context'] = self.ssl_context
        super().init_poolmanager(connections, maxsize, block, **pool_kwargs)


def download_all_main_component_data(service_key, save_filename="drug_main_component_data.json"):
    """
    '의약품 제품 주성분 상세정보' API의 모든 데이터를 안정적으로 수집하여 JSON 파일로 저장합니다.
    - 자동 재시도, 서버 부하 감소, SSL 오류 처리 기능이 포함되어 있습니다.
    """
    # --- 1. API 요청 설정 ---
    api_url_base = "https://apis.data.go.kr"
    api_path = "/1471000/DrugPrdtPrmsnInfoService06/getDrugPrdtMcpnDtlInq06" # 목표 API 경로
    full_api_url = api_url_base + api_path

    # 공공데이터포털에서 발급된 키는 URL 인코딩된 상태이므로, 디코딩하여 사용합니다.
    decoded_service_key = unquote(service_key)

    # --- 2. 안정적인 통신을 위한 세션 설정 ---
    session = requests.Session()

    # 서버 오류(5xx) 발생 시, 최대 5번까지 1초 간격으로 자동 재시도를 설정합니다.
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])

    # 위에서 정의한 CipherSuiteAdapter와 재시도 설정을 세션에 탑재합니다.
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # --- 3. 전체 데이터 수 파악 (첫 페이지 요청) ---
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status() # HTTP 에러 발생 시 예외 발생
        data = response.json()

        if data['header']['resultCode'] != '00':
            print(f"API 에러: {data['header']['resultMsg']}")
            return False

        total_count = int(data['body']['totalCount'])
        total_pages = math.ceil(total_count / rows_per_page)

        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다. (페이지당 {rows_per_page}개)")
        print("-" * 60)

    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}")
        session.close()
        return False

    # --- 4. 모든 페이지 순회하며 데이터 수집 ---
    for page in range(1, total_pages + 1):
        try:
            params = {
                'serviceKey': decoded_service_key,
                'pageNo': str(page),
                'numOfRows': str(rows_per_page),
                'type': 'json'
            }
            print(f"페이지 {page}/{total_pages} 수집 중...")

            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            # 응답이 정상이면서 'items' 데이터가 실제로 존재할 경우
            if data['header']['resultCode'] == '00' and 'items' in data['body'] and data['body']['items']:
                items_data = data['body']['items']
                all_items.extend(items_data)
            else:
                # 데이터가 없는 경우 (API의 일시적 문제 등)
                print(f"  -> 페이지 {page}에서 데이터 없음: {data['header'].get('resultMsg', '응답에 items 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이 (0.1초)
            time.sleep(0.1)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 5초간 휴식합니다...")
                time.sleep(5)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue # 실패 시 다음 페이지로 넘어감

    # --- 5. 수집 완료 후 파일로 저장 ---
    session.close()
    try:
        print("-" * 60)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")

        # UTF-8 인코딩 및 가독성을 위한 옵션으로 JSON 파일 저장
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)

        print("저장 완료! 모든 데이터 수집이 성공적으로 끝났습니다. 🎉")
        return True

    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}")
        return False

# --- 스크립트 실행 부분 ---
if __name__ == "__main__":
    # 🚨 여기에 본인의 공공데이터포털 인증키(일반 인증키, 인코딩)를 입력하세요.
    MY_SERVICE_KEY = 'k5pE3X4v37ySjqiAzhxMCS8PbYK6S7G7J8iWiiJXUs2LpCcENDTIHsQ+t5quezvkY3dgGn8OOteFJOWZkxw3Ow=='

    # 함수 실행
    download_all_main_component_data(service_key=MY_SERVICE_KEY)

데이터 수집을 시작합니다...
총 데이터: 129593개, 전체 페이지: 1296개를 수집합니다. (페이지당 100개)
------------------------------------------------------------
페이지 1/1296 수집 중...
페이지 2/1296 수집 중...
페이지 3/1296 수집 중...
페이지 4/1296 수집 중...
페이지 5/1296 수집 중...
페이지 6/1296 수집 중...
페이지 7/1296 수집 중...
페이지 8/1296 수집 중...
페이지 9/1296 수집 중...
페이지 10/1296 수집 중...
페이지 11/1296 수집 중...
페이지 12/1296 수집 중...
페이지 13/1296 수집 중...
페이지 14/1296 수집 중...
페이지 15/1296 수집 중...
페이지 16/1296 수집 중...
페이지 17/1296 수집 중...
페이지 18/1296 수집 중...
페이지 19/1296 수집 중...
페이지 20/1296 수집 중...
페이지 21/1296 수집 중...
페이지 22/1296 수집 중...
페이지 23/1296 수집 중...
페이지 24/1296 수집 중...
페이지 25/1296 수집 중...
페이지 26/1296 수집 중...
페이지 27/1296 수집 중...
페이지 28/1296 수집 중...
페이지 29/1296 수집 중...
페이지 30/1296 수집 중...
페이지 31/1296 수집 중...
페이지 32/1296 수집 중...
페이지 33/1296 수집 중...
페이지 34/1296 수집 중...
페이지 35/1296 수집 중...
페이지 36/1296 수집 중...
페이지 37/1296 수집 중...
페이지 38/1296 수집 중...
페이지 39/1296 수집 중...
페이지 40/1296 수집 중...
페이지 41/1296 수집 중...
페이지 42/1296 수집 중...
페이지 43/1296 수집 중...
페이지 44/1296 수집 중...
페

In [ ]:
import pandas as pd
import json

print("새로운 데이터('의약품 주성분 상세정보') 전처리를 시작합니다...")

# 1단계: 파일 로드 (동일)
try:
    df = pd.read_json('drug_main_component_data.json', encoding='utf-8')
    print(f"파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("drug_main_component_data.json 파일을 찾을 수 없습니다.")
    exit()

# 2단계: 기본 정제 (이전과 동일)
# ------------------------------------------------------------------------------
df.columns = [col.lower() for col in df.columns]
print("\n컬럼 이름을 소문자로 변경했습니다.")

df.drop_duplicates(subset=['item_seq', 'mtral_code'], keep='first', inplace=True)
print(f"중복 제거 후 데이터 수: {len(df)}개")

df['mtral_nm'] = df['mtral_nm'].fillna('정보 없음')
df['prduct'] = df['prduct'].fillna('제품명 없음') # 제품명 결측값 처리 추가
df['qnt'] = df['qnt'].astype(str)
print("'qnt' 컬럼을 정보 손실 없이 문자열 타입으로 변환했습니다.")

# --- [핵심 수정 부분] ---
# 3단계: 정보 그룹화 (Grouping)
# ------------------------------------------------------------------------------
print("\n'item_seq'를 기준으로 성분 정보와 제품명을 그룹화합니다...")

# 성분 정보를 딕셔너리로 만드는 함수 (이전과 동일)
def create_component_dict(row):
    return {
        'component_name': row['mtral_nm'],
        'quantity': row['qnt'],
        'unit': row['ingd_unit_cd']
    }

df['component_info'] = df.apply(create_component_dict, axis=1)

# 'item_seq'로 그룹화할 때, 'prduct' 정보와 'component_info' 정보를 함께 집계(aggregate)합니다.
# - prduct: 그룹 내 첫 번째 값을 대표로 사용 ('first')
# - component_info: 그룹 내 모든 값을 리스트로 묶음 (lambda x: list(x))
grouped_df = df.groupby('item_seq').agg(
    product_name=('prduct', 'first'),
    components_list=('component_info', lambda x: list(x))
).reset_index()

# 컬럼 이름 변경 (기존 코드와 통일성을 위해)
grouped_df.rename(columns={'product_name': 'item_name'}, inplace=True)

print("성분 정보 및 제품명 그룹화를 완료했습니다.")


# 4단계: 최종 저장
# ------------------------------------------------------------------------------
# 이제 'item_name' 컬럼이 포함된 최종 DataFrame이 생성됩니다.

# 효율적인 Parquet 포맷으로 저장 (이전과 동일)
try:
    save_filename = 'drug_data_processed_v4.parquet'
    grouped_df.to_parquet(save_filename, index=False)
    print(f"\n전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v4.csv'
    grouped_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n[경고] pyarrow 라이브러리가 없어 CSV로 저장합니다.")
    print(f"전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# 결과 확인
print("\n--- 전처리 결과 샘플 (제품명 포함) ---")
print(grouped_df.head())

새로운 데이터('의약품 주성분 상세정보') 전처리를 시작합니다...
파일 로드 성공! 원본 데이터 형태: (129593, 13)

컬럼 이름을 소문자로 변경했습니다.
중복 제거 후 데이터 수: 124158개
'qnt' 컬럼을 정보 손실 없이 문자열 타입으로 변환했습니다.

'item_seq'를 기준으로 성분 정보와 제품명을 그룹화합니다...
성분 정보 및 제품명 그룹화를 완료했습니다.

전처리 완료! 최종 데이터를 'drug_data_processed_v4.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 (제품명 포함) ---
    item_seq                                       item_name  \
0  195500005  중외5%포도당생리식염액(수출명:5%DextroseinnormalsalineInj.)   
1  195500006                                      중외5%포도당주사액   
2  195600004                                     중외20%포도당주사액   
3  195600006                                     중외50%포도당주사액   
4  195700004                                   대한포도당주사액(10%)   

                                     components_list  
0  [{'component_name': '포도당', 'quantity': '50', '...  
1  [{'component_name': '포도당', 'quantity': '50', '...  
2  [{'component_name': '포도당', 'quantity': '200', ...  
3  [{'component_name': '포도당', 'quantity': '500', ...  
4  [{'component_name': '포도당', 'qu

식품의약품안전처_의약품안전사용서비스(DUR)품목정보_병용금기 정보조회

In [ ]:
import requests
import json
import math
import time
import os
import ssl
from urllib.parse import unquote
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

class CipherSuiteAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        CIPHERS = ('ECDH+AESGCM:DH+AESGCM:ECDH+AES256:DH+AES256:ECDH+AES128:DH+AES:'
                   'ECDH+HIGH:DH+HIGH:RSA+AESGCM:RSA+AES:RSA+HIGH:!aNULL:!eNULL:!MD5:!3DES')
        self.ssl_context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        self.ssl_context.set_ciphers(CIPHERS)
        super().__init__(*args, **kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = requests.packages.urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def download_all_drug_data_in_one_go(service_key, save_filename="usjnt_taboo_info_final.json"):
    """
    일일 트래픽(10,000회)이 충분하므로, 하루 만에 모든 데이터를 수집하는 최종 함수.
    (병용금기 DUR: getUsjntTabooInfoList03)
    """
    api_url_base = "https://apis.data.go.kr"
    # ★ 병용금기 엔드포인트
    api_path = "/1471000/DURPrdlstInfoService03/getUsjntTabooInfoList03"
    full_api_url = api_url_base + api_path
    decoded_service_key = unquote(service_key)

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = CipherSuiteAdapter(max_retries=retries)
    session.mount(api_url_base, adapter)

    all_items = []
    rows_per_page = 100

    # 1. 1페이지를 요청하여 전체 데이터 수를 정확히 파악
    print("데이터 수집을 시작합니다...")
    try:
        params = {'serviceKey': decoded_service_key, 'pageNo': '1', 'numOfRows': '1', 'type': 'json'}
        response = session.get(full_api_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        # (최소 방어) response 래핑 대응
        header = data.get('header') or data.get('response', {}).get('header', {})
        body   = data.get('body')   or data.get('response', {}).get('body', {})

        if header.get('resultCode', '00') != '00':
            print(f"API 에러: {header.get('resultMsg')}")
            return False

        total_count = int(body.get('totalCount', 0))
        total_pages = math.ceil(total_count / rows_per_page)
        print(f"총 데이터: {total_count}개, 전체 페이지: {total_pages}개를 수집합니다.")
        print("-" * 50)
    except requests.exceptions.RequestException as e:
        print(f"초기 정보 파악 실패: {e}"); session.close(); return False
    except (ValueError, KeyError) as e:
        print(f"초기 응답 파싱 실패: {e}"); session.close(); return False

    # 2. 1페이지부터 마지막 페이지까지 순차적으로 수집
    for page in range(1, total_pages + 1):
        try:
            params = {'serviceKey': decoded_service_key, 'pageNo': str(page), 'numOfRows': str(rows_per_page), 'type': 'json'}
            print(f"페이지 {page}/{total_pages} 수집 중...")
            response = session.get(full_api_url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()

            header = data.get('header') or data.get('response', {}).get('header', {})
            body   = data.get('body')   or data.get('response', {}).get('body', {})

            items_data = body.get('items')
            # (보강) {"items":{"item":[...]}} 또는 dict 단건 형태도 처리
            if isinstance(items_data, dict) and 'item' in items_data:
                items_data = items_data['item']
            elif isinstance(items_data, dict):
                items_data = [items_data]

            if header.get('resultCode', '00') == '00' and items_data:
                if not isinstance(items_data, list): items_data = [items_data]
                all_items.extend(items_data)
            else:
                 print(f"  -> 페이지 {page}에서 데이터 없음: {header.get('resultMsg', '데이터 없음')}")

            # 서버 부하를 줄이기 위한 최소한의 딜레이
            time.sleep(0.2)

            # 100 페이지마다 긴 휴식을 주어 안정성 확보
            if page % 100 == 0 and page != total_pages:
                print(f"  -> {page} 페이지 도달. 서버 부하 방지를 위해 10초간 휴식합니다...")
                time.sleep(10)

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"  -> !!! 페이지 {page} 수집 실패: {e}. 이 페이지를 건너뜁니다.")
            continue

    # 3. 최종 저장
    session.close()
    try:
        print("-" * 50)
        print(f"총 {len(all_items)}개의 데이터를 성공적으로 수집했습니다.")
        print(f"'{save_filename}' 파일로 최종 저장합니다...")
        with open(save_filename, 'w', encoding='utf-8') as f:
            json.dump(all_items, f, ensure_ascii=False, indent=2)
        print(f"저장 완료! 모든 데이터 수집이 완료되었습니다!")
        return True
    except Exception as e:
        print(f"최종 파일 저장 중 에러 발생: {e}"); return False

# --- 스크립트 실행 ---
if __name__ == "__main__":
    # 🔑 본인 '인증키(Decoding)'를 넣으세요
    MY_SERVICE_KEY = "3CvsJlAuubt7ixPbs8bPlxXgqa7XndhJTeJpI7VJ2v1Wu7keM1MBQO2FxcoaEUNwF8ADXxzGniiuKweWsDo5hQ=="
    download_all_drug_data_in_one_go(service_key=MY_SERVICE_KEY)

데이터 수집을 시작합니다...
총 데이터: 237454개, 전체 페이지: 2375개를 수집합니다.
--------------------------------------------------
페이지 1/2375 수집 중...
페이지 2/2375 수집 중...
페이지 3/2375 수집 중...
페이지 4/2375 수집 중...
페이지 5/2375 수집 중...
페이지 6/2375 수집 중...
페이지 7/2375 수집 중...
페이지 8/2375 수집 중...
페이지 9/2375 수집 중...
페이지 10/2375 수집 중...
페이지 11/2375 수집 중...
페이지 12/2375 수집 중...
페이지 13/2375 수집 중...
페이지 14/2375 수집 중...
페이지 15/2375 수집 중...
페이지 16/2375 수집 중...
페이지 17/2375 수집 중...
페이지 18/2375 수집 중...
페이지 19/2375 수집 중...
페이지 20/2375 수집 중...
페이지 21/2375 수집 중...
페이지 22/2375 수집 중...
페이지 23/2375 수집 중...
페이지 24/2375 수집 중...
페이지 25/2375 수집 중...
페이지 26/2375 수집 중...
페이지 27/2375 수집 중...
페이지 28/2375 수집 중...
페이지 29/2375 수집 중...
페이지 30/2375 수집 중...
페이지 31/2375 수집 중...
페이지 32/2375 수집 중...
페이지 33/2375 수집 중...
페이지 34/2375 수집 중...
페이지 35/2375 수집 중...
페이지 36/2375 수집 중...
페이지 37/2375 수집 중...
페이지 38/2375 수집 중...
페이지 39/2375 수집 중...
페이지 40/2375 수집 중...
페이지 41/2375 수집 중...
페이지 42/2375 수집 중...
페이지 43/2375 수집 중...
페이지 44/2375 수집 중...
페이지 45/2375 수집 중...
페이지

In [ ]:
import pandas as pd
import json

print("병용금기(DUR) 데이터 전처리를 시작합니다...")

# --- 1단계: 파일 로드 ---
try:
    df = pd.read_json('usjnt_taboo_info_final.json', encoding='utf-8')
    print(f"✅ 파일 로드 성공! 원본 데이터 형태: {df.shape}")
except FileNotFoundError:
    print("❌ 'usjnt_taboo_info_final.json' 파일을 찾을 수 없습니다.")
    exit()

# --- 2단계: 기본 정제 ---
df.columns = [col.lower() for col in df.columns]
print("\n🔄 컬럼 이름을 소문자로 변경했습니다.")

before_dedup = len(df)
df.drop_duplicates(subset=['item_name', 'mixture_ingr_kor_name'], keep='first', inplace=True)
after_dedup = len(df)
print(f"🗑️ 중복 제거: {before_dedup}개 -> {after_dedup}개 ({before_dedup - after_dedup}개 제거)")

text_cols = ['prohbt_content', 'item_name', 'mixture_ingr_kor_name', 'item_seq']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna('')
print("🧹 결측값 처리를 완료했습니다.")

# --- 3단계: 정보 구조화 ---
print("\n⚙️  'item_name'을 기준으로 병용금기 정보를 그룹화합니다...")

def create_interaction_dict(row):
    return {
        'prohibited_ingredient': row['mixture_ingr_kor_name'],
        'reason': row['prohbt_content']
    }
df['interaction_info'] = df.apply(create_interaction_dict, axis=1)

grouped_df = df.groupby('item_name').agg(
    item_seq=('item_seq', 'first'),
    interactions_list=('interaction_info', list)
).reset_index()

print("✅ 정보 그룹화를 완료했습니다.")

# --- 4단계: 최종 저장 ---
final_df = grouped_df[['item_seq', 'item_name', 'interactions_list']]

# [핵심 수정] 나중에 통합할 때 이름이 충돌하지 않도록 미리 컬럼 이름을 변경합니다.
final_df.rename(columns={'interactions_list': 'interactions_list_2'}, inplace=True)
print("\n✨ 'interactions_list' 컬럼을 'interactions_list_2'로 변경했습니다.")

try:
    save_filename = 'drug_data_processed_v5.parquet'
    final_df.to_parquet(save_filename, index=False)
    print(f"\n💾 전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")
except ImportError:
    save_filename = 'drug_data_processed_v5.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 전처리 완료! 최종 데이터를 '{save_filename}' 파일로 저장했습니다.")

# --- 결과 확인 ---
print("\n--- 전처리 결과 샘플 ---")
print(final_df.head().to_string())

병용금기(DUR) 데이터 전처리를 시작합니다...
✅ 파일 로드 성공! 원본 데이터 형태: (237454, 43)

🔄 컬럼 이름을 소문자로 변경했습니다.
🗑️ 중복 제거: 237454개 -> 15506개 (221948개 제거)
🧹 결측값 처리를 완료했습니다.

⚙️  'item_name'을 기준으로 병용금기 정보를 그룹화합니다...
✅ 정보 그룹화를 완료했습니다.

✨ 'interactions_list' 컬럼을 'interactions_list_2'로 변경했습니다.

💾 전처리 완료! 최종 데이터를 'drug_data_processed_v5.parquet' 파일로 저장했습니다.

--- 전처리 결과 샘플 ---
    item_seq                         item_name                                                                                                                                                                                                                                                                           interactions_list_2
0  201706222  가나플럭스정20/1100밀리그램(오메프라졸,탄산수소나트륨)                                                                                                                                                                                                   [{'prohibited_ingredient': '릴피비린염산염', 'reason': '위장 pH의 증가로 릴피비린의 흡수가 저하되어 릴피비린

In [ ]:
import pandas as pd
import os
import ast
import numpy as np

print("의약품 데이터 통합 작업을 시작합니다. 🎉 (총 4개 파일, 상호작용 정보 분리 저장)")

# --- 1. 파일 경로 정의 ---
file_v2 = 'drug_data_processed_v2.parquet'
file_v3 = 'drug_data_processed_v3.parquet'
file_v4 = 'drug_data_processed_v4.parquet'
file_v5 = 'drug_data_processed_v5.parquet' # 이 파일은 'interactions_list_2' 컬럼을 가짐

# --- 2. 데이터 로드 및 Key 표준화 ---
def load_and_standardize(file_path):
    """
    Parquet 또는 CSV 파일을 로드하고, 'item_seq'를 문자열 형태로 표준화합니다.
    """
    csv_path = file_path.replace('.parquet', '.csv')
    try:
        df = None
        if os.path.exists(file_path):
            df = pd.read_parquet(file_path)
        elif os.path.exists(csv_path):
            df = pd.read_csv(csv_path, dtype={'item_seq': str})
        else:
            print(f"❌ '{file_path}' 파일을 찾을 수 없습니다.")
            exit()

        if 'item_seq' in df.columns:
            df.dropna(subset=['item_seq'], inplace=True)
            df['item_seq'] = pd.to_numeric(df['item_seq'], errors='coerce').fillna(0).astype(np.int64).astype(str)

        return df
    except Exception as e:
        print(f"❌ '{file_path}' 파일 로드 중 에러 발생: {e}")
        exit()

df_details = load_and_standardize(file_v2)
df_info = load_and_standardize(file_v3)
df_components = load_and_standardize(file_v4)
df_interaction = load_and_standardize(file_v5)

print("\n데이터 로드 완료! 데이터 병합을 시작합니다...")

# --- 3. 데이터 병합 (Merge) ---
# v2, v3, v4를 먼저 병합합니다.
merged_base = pd.merge(df_details, df_info, on='item_seq', how='outer', suffixes=('_v2', '_v3'))
merged_base = pd.merge(merged_base, df_components, on='item_seq', how='outer')
print(f"\n1차 병합(v2+v3+v4) 후: {merged_base.shape}")

# v5를 병합합니다. 'interactions_list'와 'interactions_list_2'는 이름이 달라 충돌 없이 각각의 컬럼으로 생성됩니다.
final_df = pd.merge(merged_base, df_interaction, on='item_seq', how='left')
print(f"병용금기 정보 병합 후: {final_df.shape}")


# --- 4. 중복 컬럼 정리 ---
print("\n중복된 컬럼들을 정리합니다...")

# item_name 및 나머지 중복/임시 컬럼 정리
final_df['item_name'] = final_df['item_name_v2'].fillna(final_df['item_name_v3']).fillna(final_df.get('item_name_x')).fillna(final_df.get('item_name_y'))
if 'item_name_clean_v2' in final_df.columns:
    final_df['item_name_clean'] = final_df['item_name_clean_v2'].fillna(final_df.get('item_name_clean_v3'))
if 'entp_name_v2' in final_df.columns:
    final_df['entp_name'] = final_df['entp_name_v2'].fillna(final_df.get('entp_name_v3'))

# 임시 컬럼 삭제
cols_to_drop = [col for col in final_df.columns if any(s in col for s in ['_v2', '_v3', '_x', '_y'])]
final_df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print("컬럼 정리 완료!")

# 최종 컬럼 순서 재배치 (두 개의 interactions_list를 모두 앞으로)
first_cols = ['item_seq', 'item_name', 'entp_name', 'interactions_list', 'interactions_list_2']
other_cols = [col for col in final_df.columns if col not in first_cols]
final_df = final_df[[col for col in first_cols if col in final_df.columns] + other_cols]

print(f"최종 데이터 형태: {final_df.shape}")
print("\n최종 컬럼 목록:", final_df.columns.tolist())

# --- 5. 최종 데이터 저장 ---
try:
    save_filename = 'drug_data_integrated_final.parquet'
    final_df.to_parquet(save_filename, index=False, compression='gzip')
    print(f"\n💾 모든 데이터가 통합되었습니다! 최종 파일 '{save_filename}'(으)로 저장 완료!")
except:
    save_filename = 'drug_data_integrated_final.csv'
    final_df.to_csv(save_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 모든 데이터가 통합되었습니다! 최종 파일 '{save_filename}'(으)로 저장 완료!")

# --- 결과 확인 ---
print("\n--- 통합 결과 샘플 (병용금기 정보 포함) ---")
# 두 리스트 중 하나라도 내용이 있는 경우를 샘플로 선택
sample_df = final_df[
    final_df['interactions_list'].notna() | final_df['interactions_list_2'].notna()
]

if not sample_df.empty:
    display_cols = ['item_name', 'interactions_list', 'interactions_list_2']
    pd.set_option('display.max_colwidth', 80)
    print(sample_df[display_cols].head(10).to_string())
else:
    print("병용금기 정보가 포함된 샘플을 찾지 못했습니다.")

의약품 데이터 통합 작업을 시작합니다. 🎉 (총 4개 파일, 상호작용 정보 분리 저장)

데이터 로드 완료! 데이터 병합을 시작합니다...

1차 병합(v2+v3+v4) 후: (77433, 22)
병용금기 정보 병합 후: (77433, 24)

중복된 컬럼들을 정리합니다...
컬럼 정리 완료!
최종 데이터 형태: (77433, 19)

최종 컬럼 목록: ['item_seq', 'item_name', 'entp_name', 'interactions_list', 'interactions_list_2', 'etc_otc_code', 'ingr_name', 'ingredients_list', 'ee_doc_data', 'ud_doc_data', 'nb_doc_data', 'spclty_pblc', 'prduct_type', 'item_ingr_name', 'main_ingredients_list', 'big_prdt_img_url', 'edi_code', 'components_list', 'item_name_clean']

💾 모든 데이터가 통합되었습니다! 최종 파일 'drug_data_integrated_final.parquet'(으)로 저장 완료!

--- 통합 결과 샘플 (병용금기 정보 포함) ---
                                        item_name                                                                  interactions_list                                                                                                                                                                                                                                                    

In [13]:
pip install pandas pyarrow sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.2 MB/s eta 0:00:00


챗봇

In [ ]:
import pandas as pd
import os
import re
import numpy as np
import faiss
import html
from sentence_transformers import SentenceTransformer

# ==========================
# 0) 텍스트 정리/추출 유틸
# ==========================

GENERIC_TERMS = ["효능", "부작용", "상호작용", "용법", "용량", "주의", "경고", "금기", "성분", "가격", "복용", "처방"]

def clean_html_tags(text: str) -> str:
    """태그/CDATA 제거 + HTML 엔티티 복원 + 공백 정리"""
    if not isinstance(text, str):
        return ""
    # CDATA -> 내용만 보존
    text = re.sub(r"<!\[CDATA\[(.*?)\]\]>", r"\1", text, flags=re.DOTALL)
    # 태그 제거
    text = re.sub(r"<[^>]+>", " ", text)
    # HTML 엔티티 복원
    text = html.unescape(text)
    # 공백 정리
    text = re.sub(r"[ \t\u00A0]+", " ", text).strip()
    # 과도한 구분선 제거
    lines = [l.strip() for l in text.splitlines() if not re.fullmatch(r"[-_=]{5,}", l.strip())]
    return "\n".join([l for l in lines if l])

def split_sentences(text: str):
    """한/영 혼용 문장 분리(간단 버전)"""
    if not text:
        return []
    text = re.sub(r"\s+", " ", text)
    sents = re.split(r"(?<=[\.!?])\s+|(?<=[\u3002\uFF01\uFF1F])\s+|(?<=])\s+", text)
    return [s.strip() for s in sents if s and len(s.strip()) > 1]

def extract_by_keywords(text: str, keywords, max_n=5):
    """키워드가 포함된 문장만 추출 (중복 제거)"""
    sents = split_sentences(text)
    picks = []
    seen = set()
    for s in sents:
        low = s.lower()
        if any(k.lower() in low for k in keywords):
            if s not in seen:
                picks.append(s)
                seen.add(s)
        if len(picks) >= max_n:
            break
    return picks

def normalize_query(q: str) -> dict:
    """
    질의에서 제품명 매칭용 토큰과 의미검색용 질의를 동시에 준비.
    - 제품명 매칭: 일반 용어 제거 후 남은 토큰들로 contains OR 검색
    - 의미검색: 원 질의에서 일반 용어만 제거
    """
    q = (q or "").strip()
    # 제품명 매칭 토큰: 일반 용어 제거하고 공백/구두점 기준 분리
    toks = re.split(r"[,\s/|;·]+", q)
    toks = [t for t in toks if t and t not in GENERIC_TERMS]
    # 한 글자 토큰은 과매칭 유발 -> 2자 이상만
    name_tokens = [t for t in toks if len(t) >= 2]

    # 의미검색용 질의(부작용/상호작용 같은 일반 단어 제거)
    simplified = q
    for g in GENERIC_TERMS:
        simplified = simplified.replace(g, " ")
    simplified = re.sub(r"\s+", " ", simplified).strip() or q
    return {"name_tokens": name_tokens, "semantic_query": simplified}

# ==========================
# 1) 데이터 준비
# ==========================

def load_and_prepare_data(data_path='drug_data_integrated_final.parquet'):
    print("1. 데이터 파일을 로드하고 정리합니다...")
    if not os.path.exists(data_path):
        print(f"'{data_path}' 파일이 없습니다.")
        return None
    df = pd.read_parquet(data_path)

    # 안전하게 문자열화 + 정리
    def s(x):
        return clean_html_tags(str(x)) if not isinstance(x, list) else clean_html_tags(", ".join(map(str, x)))

    # 검색용 텍스트: 제품명/회사/성분 + 효능/용법/주의/상호작용 등 가급적 많이 포함
    df['__ee'] = df.get('ee_doc_data', '').apply(s) if 'ee_doc_data' in df.columns else ""
    df['__ud'] = df.get('ud_doc_data', '').apply(s) if 'ud_doc_data' in df.columns else ""
    df['__nb'] = df.get('nb_doc_data', '').apply(s) if 'nb_doc_data' in df.columns else ""
    df['__inter1'] = df.get('interactions_list', '').apply(s) if 'interactions_list' in df.columns else ""
    df['__inter2'] = df.get('interactions_list_2', '').apply(s) if 'interactions_list_2' in df.columns else ""
    df['__ingr'] = df.get('ingr_name', '').apply(s) if 'ingr_name' in df.columns else ""
    df['__entp'] = df.get('entp_name', '').apply(s) if 'entp_name' in df.columns else ""
    df['__item'] = df.get('item_name', '').apply(s) if 'item_name' in df.columns else ""

    df['search_text'] = (
        "제품명: " + df['__item'] + ". "
        "제조사: " + df['__entp'] + ". "
        "성분: " + df['__ingr'] + ". "
        "효능: " + df['__ee'] + ". "
        "용법: " + df['__ud'] + ". "
        "주의/경고: " + df['__nb'] + ". "
        "상호작용: " + df['__inter1'] + " " + df['__inter2']
    )

    # 너무 짧은 레코드는 제거
    df = df[df['search_text'].str.len() > 50].reset_index(drop=True)
    return df

# ==========================
# 2) FAISS 인덱스
# ==========================

def create_vector_index(df, model):
    print("2. 의약품 데이터를 벡터로 변환합니다. (이 과정이 '학습'에 해당합니다)")
    embeddings = model.encode(df['search_text'].tolist(), show_progress_bar=True)
    print("3. FAISS 인덱스를 생성하여 벡터를 저장합니다...")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index

# ==========================
# 3) 검색
# ==========================

def search_semantic(query, model, index, df, k=1):
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, k)
    if indices.size > 0:
        best_match_index = indices[0][0]
        return df.iloc[best_match_index]
    return None

def keyword_match(df, name_tokens):
    """제품명/성분명 등에서 토큰 OR 매칭"""
    if not name_tokens:
        return pd.DataFrame()
    # 안전한 정규식 OR
    pat = "|".join([re.escape(t) for t in name_tokens])
    m_item = df['__item'].str.contains(pat, case=False, na=False)
    m_ingr = df['__ingr'].str.contains(pat, case=False, na=False)
    m_entp = df['__entp'].str.contains(pat, case=False, na=False)
    matches = df[m_item | m_ingr | m_entp]
    return matches

# ==========================
# 4) 결과 포맷팅
# ==========================

ADVERSE_KWS = ["부작용", "이상반응", "유해사례", "불편", "adverse", "side effect"]
INTERACT_KWS = ["상호작용", "병용", "병용투여", "금기 병용", "interact", "interaction", "병용 시", "함께 복용"]

def build_output_sections(row, max_len=800):
    # 원문 텍스트들
    eff = row.get('__ee', '')
    ud  = row.get('__ud', '')
    nb  = row.get('__nb', '')
    inter1 = row.get('__inter1', '')
    inter2 = row.get('__inter2', '')

    # 부작용/상호작용 후보 텍스트 풀
    adverse_src = " ".join([nb, eff, ud])  # 설명서의 주의/경고/효능/용법에서도 흔히 언급됨
    interaction_src = " ".join([inter1, inter2, nb, eff, ud])

    adverse_lines = extract_by_keywords(adverse_src, ADVERSE_KWS, max_n=5)
    if not adverse_lines and nb:
        # 키워드가 빠진 경우를 대비해 주의문 일부 제공
        adverse_lines = split_sentences(nb)[:3]

    interaction_lines = extract_by_keywords(interaction_src, INTERACT_KWS, max_n=5)
    if not interaction_lines and (inter1 or inter2):
        # 전용 상호작용 컬럼이 있으면 거기서 앞부분만
        interaction_lines = split_sentences(inter1 + " " + inter2)[:3]

    # 효능은 ee에서 가져오되 길이 제한
    efficacy = eff[:max_len] + ("..." if len(eff) > max_len else "")
    return efficacy, adverse_lines, interaction_lines

# ==========================
# 5) 메인 루프
# ==========================

def main():
    df = load_and_prepare_data()
    if df is None:
        return

    print("임베딩 모델을 로드합니다... (최초 실행 시 다운로드에 시간이 걸릴 수 있습니다)")
    embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
    vector_index = create_vector_index(df, embedding_model)

    print("\n" + "="*50)
    print("안녕하세요! 하이브리드 검색 의약품 챗봇입니다.")
    print("제품명 또는 증상을 질문해주세요. (종료하려면 '종료')")
    print("="*50)

    while True:
        try:
            user_query = input("\n> 질문: ").strip()
            if user_query == "종료":
                print("챗봇을 종료합니다.")
                break
            if not user_query:
                continue

            # 1) 질의 정규화
            qinfo = normalize_query(user_query)

            # 2) 제품명/성분 키워드 매칭 우선
            result = None
            kw_matches = keyword_match(df, qinfo["name_tokens"])
            if not kw_matches.empty:
                print("\n[제품명/성분 기반 검색 결과]")
                # 매칭이 여러 개면 제품명 길이/정확도 기준 우선(간단히 첫 행)
                result = kw_matches.iloc[0]
            else:
                # 3) 의미(임베딩) 기반 검색
                print("\n[증상/효능 기반 AI 검색 결과]")
                result = search_semantic(qinfo["semantic_query"], embedding_model, vector_index, df)

            # --- 결과 출력 ---
            if result is not None:
                print(f"제품명: {result.get('__item','')}")
                print(f"제조사: {result.get('__entp','')}")
                print("-" * 30)

                efficacy, adverse_lines, interaction_lines = build_output_sections(result)

                print("효능:")
                print(efficacy if efficacy else "(정보 없음)")
                print()

                print("부작용(요약):")
                if adverse_lines:
                    for s in adverse_lines:
                        print(f"- {s}")
                else:
                    print("(정보 없음)")
                print()

                print("상호작용(요약):")
                if interaction_lines:
                    for s in interaction_lines:
                        print(f"- {s}")
                else:
                    print("(정보 없음)")

            else:
                print("관련 정보를 찾지 못했습니다.")

        except Exception as e:
            print("\n[오류 발생] 답변 처리 중 문제가 발생했습니다.")
            print(f"오류 내용: {e}")
            print("다른 질문을 시도해주세요.")

if __name__ == "__main__":
    main()


1. 데이터 파일을 로드하고 정리합니다...
임베딩 모델을 로드합니다... (최초 실행 시 다운로드에 시간이 걸릴 수 있습니다)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2. 의약품 데이터를 벡터로 변환합니다. (이 과정이 '학습'에 해당합니다)
